In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [29]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier, Pool
from transformers import AutoTokenizer, AutoModel


import wandb

## Загрузка данных

In [43]:
np.random.seed(59)
X_train = pd.read_csv("data/my/before_feature_engineering/train.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
y_train = pd.read_csv("data/my/before_feature_engineering/train_labels.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
X_train_tr = pd.read_csv("data/my/after_feature_engineering/train.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
groups_train = pd.read_csv("data/my/before_feature_engineering/train_groups.csv", sep=",", header=0).drop(columns="Unnamed: 0")

permutation = np.random.permutation(X_train.shape[0])
X_train = X_train.iloc[permutation].reset_index(drop=True)
X_train_tr = X_train_tr.iloc[permutation].reset_index(drop=True)
y_train = y_train.iloc[permutation].reset_index(drop=True)
groups_train = groups_train.iloc[permutation].reset_index(drop=True)["0"]

X_test = pd.read_csv("data/my/before_feature_engineering/test.csv", sep=",", header=0).drop(columns="Unnamed: 0")
X_test_tr = pd.read_csv("data/my/after_feature_engineering/test.csv", sep=",", header=0).drop(columns="Unnamed: 0")
y_test = pd.read_csv("data/my/before_feature_engineering/test_labels.csv", sep=",", header=0).drop(columns="Unnamed: 0")

y_train = y_train.fraudulent
y_test = y_test.fraudulent

## Эксперименты с моделями без тюнинга гиперпараметров

### Вспомогательные функции

In [69]:

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        self.estimators = estimators

    def predict(self, X):
        predictions = np.array([estimator.predict(X) for estimator in self.estimators])
        sum_predictions = np.sum(predictions.T, axis=1)
        majority = (sum_predictions > (len(self.estimators) / 2)).astype(int)
        return majority

def cross_validate_and_log_metrics(model, X, y, groups, cv_class=StratifiedGroupKFold, n_splits=5, test_share_range=(0.1, 0.3)):
    cv = cv_class(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = {'f1': [], 'precision': [], 'recall': []}
    models = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups=groups)):
        if isinstance(X, pd.DataFrame):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        elif isinstance(X, np.ndarray):
            X_train, X_val = X[train_idx], X[val_idx]
        else:
            raise TypeError('Invalid type for X argument')
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        elif isinstance(y, np.ndarray):
            y_train, y_val = y[train_idx], y[val_idx]
        else:
            raise TypeError('Invalid type for y argument')
        test_share = X_val.shape[0] / (X_val.shape[0] + X_train.shape[0])
        if test_share < test_share_range[0] or test_share > test_share_range[1]:
            continue
    
        model_new = clone(model)
        model_new.fit(X_train, y_train)
        models.append(model_new)
        y_pred = model_new.predict(X_val)
        
        fold_metrics['f1'].append(f1_score(y_val, y_pred, zero_division=0))
        fold_metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
        fold_metrics['recall'].append(recall_score(y_val, y_pred, zero_division=0))
        
        wandb.log({
            'fold': fold + 1,
            'fold_f1': fold_metrics['f1'][-1],
            'fold_precision': fold_metrics['precision'][-1],
            'fold_recall': fold_metrics['recall'][-1]
        })
    
    mean_metrics = {
        'mean_f1': np.mean(fold_metrics['f1']),
        'mean_precision': np.mean(fold_metrics['precision']),
        'mean_recall': np.mean(fold_metrics['recall'])
    }
    
    wandb.log({
        **mean_metrics,
        'status': 'completed'
    })
    
    return mean_metrics, MajorityVoteClassifier(models)

def calc_and_print_metrics(model, X, y, is_test=True):
    y_pred = model.predict(X)
    calc_set_string = "Test" if is_test else "Train"
    print(f"{calc_set_string} F1-Score: {f1_score(y, y_pred):.3f}")
    print(f"{calc_set_string} Precision: {precision_score(y, y_pred):.3f}")
    print(f"{calc_set_string} Recall: {recall_score(y, y_pred):.3f}")
            

### Эксперименты

#### SVM

В качестве бейзлайна, мы брали логистическую регрессию, поэтому в качестве первой модели сейчас можно попробовать ядровой SVM, так как он позволит уловить нелинейные закономерности в данных за счет нелинейности в ядре.

In [48]:
model_svm = SVC(
    class_weight='balanced',
    random_state=42,
    max_iter=5000,
    kernel="rbf"
)

n_folds = 5

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "svm",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_svm = cross_validate_and_log_metrics(model_svm, X_train_tr, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_svm, X_train_tr, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_svm, X_test_tr, y_test)

Train F1-Score: 0.630
Train Precision: 0.460
Train Recall: 1.000
Test F1-Score: 0.277
Test Precision: 0.528
Test Recall: 0.188


fold,▁▅█
fold_f1,█▁▆
fold_precision,▄▁█
fold_recall,█▂▁
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.34286
fold_precision,0.75
fold_recall,0.22222


Скрины метрик (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/svm/f1.png)
![](images/svm/precision.png)
![](images/svm/recall.png)

Видно, что модель перформит лучше бейзлайна (f1_score 0.27 против 0.23) -- нелинейности помогли

#### Случайаный лес

Теперь попробуем ещё усложнить модель, взяв случайный лес -- может быть, качество еще вырастет.

In [49]:
model_rf = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
)

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "random_forest",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_rf = cross_validate_and_log_metrics(model_rf, X_train_tr, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_rf, X_train_tr, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_rf, X_test_tr, y_test)

Train F1-Score: 1.000
Train Precision: 1.000
Train Recall: 1.000
Test F1-Score: 0.112
Test Precision: 1.000
Test Recall: 0.059


fold,▁▅█
fold_f1,█▃▁
fold_precision,▁▁▁
fold_recall,█▂▁
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.10526
fold_precision,1
fold_recall,0.05556


Скрины метрик (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/random_forest/f1.png)
![](images/random_forest/precision.png)
![](images/random_forest/recall.png)

Однако видно, что модель сильно переобучилась и показывает себя хуже себя, чем логистическая регрессия (бейзлайн). Может быть, тюнинг гиперпараметров поможет её регулязировать. Хотя у этой модели есть небольшой плюс -- у нее точность 100% на трейне и тесте. Она выявляет всего 5% фрода, но зато не даёт ложно-положительных результатов.

#### Градиентный бустинг

Известно, что catboost хорошо работает с текстовыми признаками, поэтому обучим его на исходных текстовых признаках после их предобработки. За счет нативной работы с текстовыми признаками, мы ожидаем, что результат может потенциально превзойти предудщие эксперименты по качеству.

In [ ]:
raw_text_features = ["title_processed", "description_processed", "company_profile_processed"]
X_train_cb = X_train[raw_text_features].fillna('')
X_test_cb = X_test[raw_text_features].fillna('')

In [112]:
class MetricsLoggerCallback:
    def after_iteration(self, info):
        test_metrics = info.metrics["validation"]
        wandb.log({
            'f1': test_metrics['F1'][-1],
            'precision': test_metrics['Precision:use_weights=false'][-1],
            'recall': test_metrics['Recall:use_weights=false'][-1],
        })
        return True

In [113]:
model_cb = CatBoostClassifier(
        iterations=100,  
        loss_function='Logloss',
        eval_metric='F1',
        custom_metric=['Precision', 'Recall'],
        class_weights=[1, 5],
        text_features=raw_text_features,
        verbose=False,
        use_best_model=False,
        early_stopping_rounds=None,
        
    )


with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "catboost",
          }):
    model_cb.fit(
        X_train_cb, y_train, 
        callbacks=[MetricsLoggerCallback()],
        eval_set=[Pool(X_test_cb, y_test, text_features=raw_text_features)]
    )
    calc_and_print_metrics(model_cb, X_train_cb, y_train, is_test=False)
    calc_and_print_metrics(model_cb, X_test_cb, y_test)


Train F1-Score: 0.801
Train Precision: 0.705
Train Recall: 0.928
Test F1-Score: 0.446
Test Precision: 1.000
Test Recall: 0.287


f1,▁▁██▇▇▇▇▆▁▃▃▃▃▃▅▅▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▄▅▄▅▄
precision,▁▄██████████████████████████████████████
recall,▁█▅▇▇▂▂▅▃▃▁▅▅▅▅▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▄▅▄▅▄▄
f1,0.44615
precision,1
recall,0.28713


Скрины метрик на тесте:

![](images/catboost/f1.png)
![](images/catboost/precision.png)
![](images/catboost/recall.png)

Видно, что модель переобучилась, но при этом качество все равно существенно улучшилось по сравнению со всеми предыдущими экспериментами.

#### BERT + логистическая регрессия

Ранее мы обрабатывали текст "классическими" методами (tf-idf, эвристики катбуста). Кажется, что контекстуальный эмбеддинг BERTа будет менее зашумлен и нести больше информации о тексте, что нам может помочь улучшить качество детекции фрода.

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def get_single_embedding(text):
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state[:, 0].cpu().numpy()

def get_all_embeddings(df):
    title_embs = []
    desc_embs = []
    profile_embs = []
    
    for _, row in df.iterrows():
        title_embs.append(get_single_embedding(str(row['title'])))
        desc_embs.append(get_single_embedding(str(row['description'])))
        profile_embs.append(get_single_embedding(str(row['company_profile'])))
    
    # Concatenate embeddings and return as numpy array
    return np.hstack([
        np.vstack(title_embs),
        np.vstack(desc_embs),
        np.vstack(profile_embs)
    ])

preprocessor = StandardScaler()
X_train_embeds = pd.DataFrame(preprocessor.fit_transform(get_all_embeddings(X_train)))
X_test_embeds = pd.DataFrame(preprocessor.transform(get_all_embeddings(X_test)))

/home/aksenovan/my_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
model_bert_logreg = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "bert+logreg",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_bert_logreg = cross_validate_and_log_metrics(model_bert_logreg, X_train_embeds, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_bert_logreg, X_train_embeds, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_bert_logreg, X_test_embeds, y_test)

Train F1-Score: 1.000
Train Precision: 1.000
Train Recall: 1.000
Test F1-Score: 0.512
Test Precision: 0.620
Test Recall: 0.436


fold,▁▅█
fold_f1,▁█▆
fold_precision,▁▇█
fold_recall,▁█▃
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.12281
fold_precision,0.11667
fold_recall,0.12963


Скрины метрик на валидации (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/bert/f1.png)
![](images/bert/precision.png)
![](images/bert/recall.png)

Видно, что качество на тесте существенно увеличилось. Однако отметим, что качество на валидации очень низкое -- думаю, это связано с утечкой в данных. Скорее всего фродеры составляют объявления по некоторму шаблону, в результате объявления одного фродера очень близки в пространстве эмбеддингов берта. В результате в тесте модель видит похожий эмбеддинг и выдает ответ, который она видела в трейне. Чтобы это поправить, я пробовал регуляризовать модель, уменьшая размерность с помощью PCA, но это, к сожалению, не помогло.

## Эксперименты с моделями с тюнингом гиперпараметров